ASF Search and SBAS Stack Initialization

In [4]:
from datetime import date
import asf_search as asf
from pprint import pprint

stack_start = date(2023, 1, 1)
stack_end = date(2025, 6, 1)
# Define ASF search parameters
opts = asf.ASFSearchOptions(**{
    "maxResults": 5000,
    "bbox": [11.00, 44.30, 11.7, 44.7],  # Bologna
    "beamSwath": ["IW"],
    "flightDirection": "DESCENDING",  # Use DESCENDING first, see note below
    "polarization": ["VV+VH", "VV"],
    "processingLevel": ["SLC"],
    "start": stack_start.isoformat(),
    "end": stack_end.isoformat(),
    "dataset": ["SENTINEL-1"]
})

# Search SLC scenes from ASF
search_results = asf.search(opts=opts)
print(f"Found {len(search_results)} scenes.")

# Create baseline stack (SBAS-compatible) from the most recent scene
baseline_results = asf.baseline_search.stack_from_product(search_results[-1])
print(f"Generated {len(baseline_results)} SBAS pairs.")

# Preview first few SBAS pairs
pprint(baseline_results[:3])


/mnt/data/tmp/ipykernel_1773942/4234889921.py:10: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  opts = asf.ASFSearchOptions(**{
/mnt/data/tmp/ipykernel_1773942/4234889921.py:23: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  search_results = asf.search(opts=opts)


Found 164 scenes.
Generated 511 SBAS pairs.


Make some directories 

In [102]:
import os
import subprocess
import glob
import subprocess
from pathlib import Path

# Define paths
root_data = Path("/mnt/data")
project_dir  = os.path.join(root_data, "gmtsar_test_2")
data_dir = os.path.join(project_dir, "data")
reframed_dir = os.path.join(project_dir, "reframed")
pins_ll = os.path.join(reframed_dir, "pins.ll")
safe_list_path = os.path.join(data_dir, "SAFE_filelist.txt")
download_dir = data_dir
dem_dir = os.path.join(project_dir, "topo")
orbit_dir = os.path.join(project_dir, "orbit")
# Orbit files go under raw

os.makedirs(download_dir, exist_ok=True)
os.makedirs(dem_dir, exist_ok=True)

print(f"DEM directory: {dem_dir}")
print(f"RAW SLC directory: {download_dir}")
print(f"Orbit directory: {orbit_dir}")





DEM directory: /mnt/data/gmtsar_test_2/topo
RAW SLC directory: /mnt/data/gmtsar_test_2/data
Orbit directory: /mnt/data/gmtsar_test_2/orbit


Make a dataframe of search results 

In [6]:
import pandas as pd

# Convert to DataFrame
search_df = pd.DataFrame([result.properties for result in search_results])

# Check available columns
print("Available columns:\n", search_df.columns.tolist())

# Preview first few rows
search_df[['sceneName', 'platform', 'polarization','beamModeType','orbit', 'startTime']].head()


Available columns:
 ['centerLat', 'centerLon', 'stopTime', 'fileID', 'flightDirection', 'pathNumber', 'processingLevel', 'url', 'startTime', 'sceneName', 'browse', 'platform', 'bytes', 'md5sum', 'frameNumber', 'granuleType', 'orbit', 'polarization', 'processingDate', 'sensor', 'groupID', 'pgeVersion', 'fileName', 'beamModeType', 's3Urls']


,sceneName,platform,polarization,beamModeType,orbit,startTime
0,S1C_IW_SLC__1SDV_20250529T051837_20250529T0519...,Sentinel-1C,VV+VH,IW,2541,2025-05-29T05:18:37Z
1,S1C_IW_SLC__1SDV_20250529T051812_20250529T0518...,Sentinel-1C,VV+VH,IW,2541,2025-05-29T05:18:12Z
2,S1A_IW_SLC__1SDV_20250528T052749_20250528T0528...,Sentinel-1A,VV+VH,IW,59390,2025-05-28T05:27:49Z
3,S1A_IW_SLC__1SDV_20250523T051940_20250523T0520...,Sentinel-1A,VV+VH,IW,59317,2025-05-23T05:19:40Z
4,S1C_IW_SLC__1SDV_20250522T052648_20250522T0527...,Sentinel-1C,VV+VH,IW,2439,2025-05-22T05:26:48Z


In [7]:
search_df.to_csv(os.path.join(project_dir, "search_df.csv"), index=False)


In [8]:
print(f"Search returned {len(search_df)} scenes.")
print("Date range:", search_df['startTime'].min(), "to", search_df['startTime'].max())


Search returned 164 scenes.
Date range: 2023-01-03T05:27:53Z to 2025-05-29T05:18:37Z


Extract a stack product in my time span 

In [10]:
from dateutil.parser import parse as parse_date
import pandas as pd
# Extract properties and geometry from baseline results
list(scene.properties.values() for scene in baseline_results[:3])
# Create a DataFrame from the baseline results
stack = pd.DataFrame([
    list(scene.properties.values()) + [scene.geometry]
    for scene in baseline_results
    if stack_start <= parse_date(scene.properties['startTime']).date() <= stack_end
], columns=list(baseline_results[0].properties.keys()) + ['geometry'])
# Show available columns
print("Available columns:\n", stack.columns.tolist())

# Preview the first few rows
stack[['sceneName', 'startTime', 'stopTime']].head()

Available columns:
 ['centerLat', 'centerLon', 'stopTime', 'fileID', 'flightDirection', 'pathNumber', 'processingLevel', 'url', 'startTime', 'sceneName', 'browse', 'platform', 'bytes', 'md5sum', 'frameNumber', 'granuleType', 'orbit', 'polarization', 'processingDate', 'sensor', 'groupID', 'pgeVersion', 'fileName', 'beamModeType', 's3Urls', 'temporalBaseline', 'perpendicularBaseline', 'geometry']


,sceneName,startTime,stopTime
0,S1A_IW_SLC__1SDV_20230103T052753_20230103T0528...,2023-01-03T05:27:53Z,2023-01-03T05:28:19Z
1,S1A_IW_SLC__1SDV_20230115T052752_20230115T0528...,2023-01-15T05:27:52Z,2023-01-15T05:28:19Z
2,S1A_IW_SLC__1SDV_20230127T052752_20230127T0528...,2023-01-27T05:27:52Z,2023-01-27T05:28:19Z
3,S1A_IW_SLC__1SDV_20230208T052751_20230208T0528...,2023-02-08T05:27:51Z,2023-02-08T05:28:18Z
4,S1A_IW_SLC__1SDV_20230220T052751_20230220T0528...,2023-02-20T05:27:51Z,2023-02-20T05:28:18Z


In [11]:
# Check available columns
print("Available columns:\n", stack.columns.tolist())

# Loop through and print geometry WKT or coordinates
for i, geom in enumerate(stack['geometry']):
    print(f"--- Geometry {i+1} ---")
    print(geom)


Available columns:
 ['centerLat', 'centerLon', 'stopTime', 'fileID', 'flightDirection', 'pathNumber', 'processingLevel', 'url', 'startTime', 'sceneName', 'browse', 'platform', 'bytes', 'md5sum', 'frameNumber', 'granuleType', 'orbit', 'polarization', 'processingDate', 'sensor', 'groupID', 'pgeVersion', 'fileName', 'beamModeType', 's3Urls', 'temporalBaseline', 'perpendicularBaseline', 'geometry']
--- Geometry 1 ---
{'coordinates': [[[11.459536, 43.427025], [11.904005, 45.044777], [8.645327, 45.445248], [8.2925, 43.827381], [11.459536, 43.427025]]], 'type': 'Polygon'}
--- Geometry 2 ---
{'coordinates': [[[11.461482, 43.426716], [11.905971, 45.044342], [8.647345, 45.444866], [8.294489, 43.827129], [11.461482, 43.426716]]], 'type': 'Polygon'}
--- Geometry 3 ---
{'coordinates': [[[11.45922, 43.427052], [11.903637, 45.044556], [8.644908, 45.445057], [8.292118, 43.827442], [11.45922, 43.427052]]], 'type': 'Polygon'}
--- Geometry 4 ---
{'coordinates': [[[11.45935, 43.426556], [11.903775, 45.044

In [12]:
# Initialize overall bounds
min_lon, min_lat = float('inf'), float('inf')
max_lon, max_lat = float('-inf'), float('-inf')

# Loop through geometries and extract bounds
for i, geom in enumerate(stack['geometry']):
    coords = geom['coordinates'][0]  # Assuming Polygon, not MultiPolygon
    print(f"--- Geometry {i+1} ---")
    for lon, lat in coords:
        print(f"Lon: {lon}, Lat: {lat}")
        # Update global bounds
        min_lon = min(min_lon, lon)
        max_lon = max(max_lon, lon)
        min_lat = min(min_lat, lat)
        max_lat = max(max_lat, lat)

# Print overall WESN bounds
print("\n[Overall Bounding Box for All Geometries]")
print(f"W: {min_lon}, E: {max_lon}, S: {min_lat}, N: {max_lat}")


--- Geometry 1 ---
Lon: 11.459536, Lat: 43.427025
Lon: 11.904005, Lat: 45.044777
Lon: 8.645327, Lat: 45.445248
Lon: 8.2925, Lat: 43.827381
Lon: 11.459536, Lat: 43.427025
--- Geometry 2 ---
Lon: 11.461482, Lat: 43.426716
Lon: 11.905971, Lat: 45.044342
Lon: 8.647345, Lat: 45.444866
Lon: 8.294489, Lat: 43.827129
Lon: 11.461482, Lat: 43.426716
--- Geometry 3 ---
Lon: 11.45922, Lat: 43.427052
Lon: 11.903637, Lat: 45.044556
Lon: 8.644908, Lat: 45.445057
Lon: 8.292118, Lat: 43.827442
Lon: 11.45922, Lat: 43.427052
--- Geometry 4 ---
Lon: 11.45935, Lat: 43.426556
Lon: 11.903775, Lat: 45.044327
Lon: 8.645387, Lat: 45.444729
Lon: 8.292593, Lat: 43.826851
Lon: 11.45935, Lat: 43.426556
--- Geometry 5 ---
Lon: 11.458768, Lat: 43.426796
Lon: 11.903153, Lat: 45.044445
Lon: 8.644742, Lat: 45.444843
Lon: 8.291981, Lat: 43.827087
Lon: 11.458768, Lat: 43.426796
--- Geometry 6 ---
Lon: 11.457108, Lat: 43.426907
Lon: 11.901499, Lat: 45.044563
Lon: 8.643207, Lat: 45.444954
Lon: 8.290436, Lat: 43.827194
Lon: 

Save a pkl of the serach object in the current pwd and in the work_dir

In [13]:
import pickle

# Save the object
with open("search_results.pkl", "wb") as f:
    pickle.dump(search_results, f)
with open(os.path.join(project_dir, "search_results.pkl"), "wb") as f:
    pickle.dump(search_results, f)
# Load it later
with open("search_results.pkl", "rb") as f:
    search_results = pickle.load(f)


Save a csv of the stack 

In [14]:
stack.to_csv(os.path.join(project_dir, "sbas_stack_metadata.csv"), index=False)


Printout the bbox based on teh geometries from stack

In [15]:
from shapely.geometry import shape
import numpy as np
import pandas as pd

# Convert geometries to shapely objects
geometries = [shape(g) for g in stack["geometry"]]
all_coords = np.vstack([np.array(geom.exterior.coords) for geom in geometries])

# Compute bounding box
min_lon, min_lat = map(float, np.min(all_coords, axis=0))
max_lon, max_lat = map(float, np.max(all_coords, axis=0))

bbox = [round(min_lon, 4), round(min_lat, 4), round(max_lon, 4), round(max_lat, 4)]

# Date range
start_dates = pd.to_datetime(stack["startTime"])
min_date = start_dates.min().date()
max_date = start_dates.max().date()

# Output
print("Clean Bounding Box:", bbox)
print("Date Range:", min_date, "to", max_date)



Clean Bounding Box: [8.1619, 42.7625, 12.1348, 46.1562]
Date Range: 2023-01-03 to 2025-05-28


Create all possible interferometric pairs

In [16]:
from itertools import combinations

# Create all possible unique interferometric pairs (master, slave)
pairs = list(combinations(stack.sort_values('startTime')['sceneName'], 2))

# Or as DataFrame for clarity
pairs_df = pd.DataFrame(pairs, columns=['master', 'slave'])

print("Total pairs:", len(pairs_df))
pairs_df.head()


Total pairs: 3003


,master,slave
0,S1A_IW_SLC__1SDV_20230103T052753_20230103T0528...,S1A_IW_SLC__1SDV_20230115T052752_20230115T0528...
1,S1A_IW_SLC__1SDV_20230103T052753_20230103T0528...,S1A_IW_SLC__1SDV_20230127T052752_20230127T0528...
2,S1A_IW_SLC__1SDV_20230103T052753_20230103T0528...,S1A_IW_SLC__1SDV_20230208T052751_20230208T0528...
3,S1A_IW_SLC__1SDV_20230103T052753_20230103T0528...,S1A_IW_SLC__1SDV_20230220T052751_20230220T0528...
4,S1A_IW_SLC__1SDV_20230103T052753_20230103T0528...,S1A_IW_SLC__1SDV_20230304T052751_20230304T0528...


Define time range and InSAR processing parameters 

In [19]:
from dateutil.parser import parse as parse_date

# Your known date range
start_date = str(min_date) # '2023-01-03'
end_date = str(max_date) # '2025-05-27'

# Parse to datetime and extract year & seasonal window
stack_start = parse_date(start_date + ' 00:00:00Z')
stack_end = parse_date(end_date + ' 00:00:00Z')

year_start = stack_start.year
year_end = stack_end.year
season_period = [start_date[5:], end_date[5:]]

# Processing parameters
max_temporal_baseline = 36              # Maximum days between pairs
interannual_baseline_max = 380          # Not applicable here, but defined
interannual_baseline_min = 350
look_nums = '20x4'                      # Range x Azimuth multilook factors
phase_filter_para = 0.8                 # Exponent for adaptive phase filtering


Filter stack for seasonal subset by year


In [20]:
from dateutil.parser import parse as parse_date

stack['startTime'] = stack['startTime'].apply(parse_date)
stack['startTime'] = stack['startTime'].dt.tz_localize(None)  # remove timezone if needed


stack_season = pd.DataFrame()
for year_index in range(int(year_start), int(year_end) + 1):
    season_start = parse_date(f'{str(year_index)}-{season_period[0]} 00:00:00Z').replace(tzinfo=None)
    season_end = parse_date(f'{str(year_index)}-{season_period[1]} 00:00:00Z').replace(tzinfo=None)
    stack_season = pd.concat([
        stack_season,
        stack.loc[(season_start <= stack.startTime) & (stack.startTime <= season_end)]
    ])


In [80]:
print("Available columns:\n", stack_season.columns.tolist())

# Preview the first few rows
stack_season[['sceneName', 'startTime', 'stopTime','temporalBaseline', 'perpendicularBaseline','geometry']].head()

Available columns:
 ['centerLat', 'centerLon', 'stopTime', 'fileID', 'flightDirection', 'pathNumber', 'processingLevel', 'url', 'startTime', 'sceneName', 'browse', 'platform', 'bytes', 'md5sum', 'frameNumber', 'granuleType', 'orbit', 'polarization', 'processingDate', 'sensor', 'groupID', 'pgeVersion', 'fileName', 'beamModeType', 's3Urls', 'temporalBaseline', 'perpendicularBaseline', 'geometry']


,sceneName,startTime,stopTime,temporalBaseline,perpendicularBaseline,geometry
0,S1A_IW_SLC__1SDV_20230103T052753_20230103T0528...,2023-01-03 05:27:53,2023-01-03T05:28:19Z,0,0.0,"{'coordinates': [[[11.459536, 43.427025], [11...."
1,S1A_IW_SLC__1SDV_20230115T052752_20230115T0528...,2023-01-15 05:27:52,2023-01-15T05:28:19Z,12,-156.0,"{'coordinates': [[[11.461482, 43.426716], [11...."
2,S1A_IW_SLC__1SDV_20230127T052752_20230127T0528...,2023-01-27 05:27:52,2023-01-27T05:28:19Z,24,35.0,"{'coordinates': [[[11.45922, 43.427052], [11.9..."
3,S1A_IW_SLC__1SDV_20230208T052751_20230208T0528...,2023-02-08 05:27:51,2023-02-08T05:28:18Z,36,-65.0,"{'coordinates': [[[11.45935, 43.426556], [11.9..."
4,S1A_IW_SLC__1SDV_20230220T052751_20230220T0528...,2023-02-20 05:27:51,2023-02-20T05:28:18Z,48,-15.0,"{'coordinates': [[[11.458768, 43.426796], [11...."


In [81]:
print("Number of SAR scenes in stack_season:", len(stack_season))
print(stack_season.head())  # default is 5 rows
print(stack_season[['sceneName', 'startTime', 'temporalBaseline']].head(10))
print(stack_season.describe(include='all'))
print(stack_season.info())


Number of SAR scenes in stack_season: 41
   centerLat  centerLon              stopTime  \
0    44.4475    10.0800  2023-01-03T05:28:19Z   
1    44.4472    10.0820  2023-01-15T05:28:19Z   
2    44.4474    10.0796  2023-01-27T05:28:19Z   
3    44.4470    10.0799  2023-02-08T05:28:18Z   
4    44.4472    10.0793  2023-02-20T05:28:18Z   

                                              fileID flightDirection  \
0  S1A_IW_SLC__1SDV_20230103T052753_20230103T0528...      DESCENDING   
1  S1A_IW_SLC__1SDV_20230115T052752_20230115T0528...      DESCENDING   
2  S1A_IW_SLC__1SDV_20230127T052752_20230127T0528...      DESCENDING   
3  S1A_IW_SLC__1SDV_20230208T052751_20230208T0528...      DESCENDING   
4  S1A_IW_SLC__1SDV_20230220T052751_20230220T0528...      DESCENDING   

   pathNumber processingLevel  \
0         168             SLC   
1         168             SLC   
2         168             SLC   
3         168             SLC   
4         168             SLC   

                                

Construct SBAS interferometric pairs - seasonal + interannual 

In [82]:
sbas_pairs = set()

print('The seasonal interferometric pairs are (reference-secondary): ')
for reference, rt in stack_season.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack_season.loc[
        (stack_season.sceneName != reference)
        & (stack_season.temporalBaseline - rt <= max_temporal_baseline)
        & (stack_season.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))
        print(reference[17:25], secondary[17:25])

print('The inter-annual interferometric pairs are (reference-secondary): ')
for reference, rt in stack_season.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack_season.loc[
        (stack_season.sceneName != reference)
        & (stack_season.temporalBaseline - rt <= interannual_baseline_max)
        & (stack_season.temporalBaseline - rt > interannual_baseline_min)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))
        print(reference[17:25], secondary[17:25])

print('The total number of interferometric pairs are: ', len(sbas_pairs))


The seasonal interferometric pairs are (reference-secondary): 
20250516 20250522
20250510 20250516
20250510 20250522
20250504 20250510
20250504 20250516
20250504 20250522
20250428 20250504
20250428 20250510
20250428 20250516
20250428 20250522
20250422 20250428
20250422 20250504
20250422 20250510
20250422 20250516
20250422 20250522
20250416 20250422
20250416 20250428
20250416 20250504
20250416 20250510
20250416 20250516
20250416 20250522
20250410 20250416
20250410 20250422
20250410 20250428
20250410 20250504
20250410 20250510
20250410 20250516
20250329 20250410
20250329 20250416
20250329 20250422
20250329 20250428
20250329 20250504
20250317 20250329
20250317 20250410
20250317 20250416
20250317 20250422
20250305 20250317
20250305 20250329
20250305 20250410
20250221 20250305
20250221 20250317
20250221 20250329
20250209 20250221
20250209 20250305
20250209 20250317
20250128 20250209
20250128 20250221
20250128 20250305
20250116 20250128
20250116 20250209
20250116 20250221
20250104 20250116
2

In [83]:
# Convert interferometric pairs set to DataFrame
ifg_pairs_df = pd.DataFrame(list(sbas_pairs), columns=["reference", "secondary"])
print(ifg_pairs_df.head())


                                           reference  \
0  S1A_IW_SLC__1SDV_20240227T052756_20240227T0528...   
1  S1A_IW_SLC__1SDV_20240403T052757_20240403T0528...   
2  S1A_IW_SLC__1SDV_20230127T052752_20230127T0528...   
3  S1A_IW_SLC__1SDV_20230220T052751_20230220T0528...   
4  S1A_IW_SLC__1SDV_20240110T052757_20240110T0528...   

                                           secondary  
0  S1A_IW_SLC__1SDV_20250305T052749_20250305T0528...  
1  S1A_IW_SLC__1SDV_20240427T052757_20240427T0528...  
2  S1A_IW_SLC__1SDV_20230220T052751_20230220T0528...  
3  S1A_IW_SLC__1SDV_20230316T052750_20230316T0528...  
4  S1A_IW_SLC__1SDV_20240122T052757_20240122T0528...  


In [84]:
## Modify the path 
ifg_pairs_df.to_csv(os.path.join(project_dir,"interferometric_pairs.csv"), index=False)


Downloading the stacks in stack_season 

In [27]:
import os
import requests
from tqdm import tqdm

# Create output folder
os.makedirs(download_dir, exist_ok=True)

# Download loop
for _, row in stack_season.iterrows():
    url = row["url"]
    filename = os.path.join(download_dir, url.split("/")[-1])

    if not os.path.exists(filename):
        print(f"Downloading {filename} ...")

        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            total_size = int(r.headers.get("Content-Length", 0))
            chunk_size = 1024 * 1024  # 1 MB

            with open(filename, 'wb') as f, tqdm(
                desc=os.path.basename(filename),
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024
            ) as bar:
                for chunk in r.iter_content(chunk_size=chunk_size):
                    f.write(chunk)
                    bar.update(len(chunk))
    else:
        print(f"Already downloaded: {filename}")


Already downloaded: /mnt/data/gmtsar_test_2/data/S1A_IW_SLC__1SDV_20230103T052753_20230103T052819_046615_059639_4587.zip
Already downloaded: /mnt/data/gmtsar_test_2/data/S1A_IW_SLC__1SDV_20230115T052752_20230115T052819_046790_059C18_D107.zip
Already downloaded: /mnt/data/gmtsar_test_2/data/S1A_IW_SLC__1SDV_20230127T052752_20230127T052819_046965_05A203_59A8.zip
Already downloaded: /mnt/data/gmtsar_test_2/data/S1A_IW_SLC__1SDV_20230208T052751_20230208T052818_047140_05A7DD_0C3D.zip
Already downloaded: /mnt/data/gmtsar_test_2/data/S1A_IW_SLC__1SDV_20230220T052751_20230220T052818_047315_05ADC6_6AC4.zip
Already downloaded: /mnt/data/gmtsar_test_2/data/S1A_IW_SLC__1SDV_20230304T052751_20230304T052818_047490_05B3B1_333C.zip
Already downloaded: /mnt/data/gmtsar_test_2/data/S1A_IW_SLC__1SDV_20230316T052750_20230316T052817_047665_05B99A_EC1E.zip
Already downloaded: /mnt/data/gmtsar_test_2/data/S1A_IW_SLC__1SDV_20230328T052751_20230328T052818_047840_05BF83_097A.zip
Already downloaded: /mnt/data/gm

S1A_IW_SLC__1SDV_20240215T052756_20240215T052823_052565_065BBA_BFC6.zip: 100%|██████████| 7.70G/7.70G [13:58<00:00, 9.85MB/s]  


S1A_IW_SLC__1SDV_20240227T052756_20240227T052823_052740_0661B3_D2A7.zip: 100%|██████████| 7.70G/7.70G [05:29<00:00, 25.1MB/s]  


S1A_IW_SLC__1SDV_20240310T052756_20240310T052823_052915_0667A2_4638.zip: 100%|██████████| 7.70G/7.70G [05:36<00:00, 24.5MB/s]  


S1A_IW_SLC__1SDV_20240322T052756_20240322T052823_053090_066E23_CB50.zip: 100%|██████████| 7.70G/7.70G [05:32<00:00, 24.9MB/s]  


...

S1C_IW_SLC__1SDV_20250428T052623_20250428T052654_002089_0045F6_764C.zip: 100%|██████████| 4.98G/4.98G [03:12<00:00, 27.8MB/s]


S1A_IW_SLC__1SDV_20250504T052750_20250504T052817_059040_0752C4_8948.zip: 100%|██████████| 4.22G/4.22G [02:56<00:00, 25.7MB/s]


S1C_IW_SLC__1SDV_20250510T052647_20250510T052714_002264_004CDC_B596.zip: 100%|██████████| 3.72G/3.72G [02:23<00:00, 27.9MB/s]


S1A_IW_SLC__1SDV_20250516T052749_20250516T052816_059215_075912_0083.zip: 100%|██████████| 4.26G/4.26G [02:46<00:00, 27.4MB/s]


S1C_IW_SLC__1SDV_20250522T052648_20250522T052715_002439_0051B0_6A61.zip: 100%|██████████| 3.94G/3.94G [02:37<00:00, 26.9MB/s]


In [ ]:
# Test and rm the corrupted SAFE 


""" 
%%bash 
# Path to the directory with SAFE folders
SAFE_DIR="/mnt/data/gmtsar_test_1/data"

# Loop through each .SAFE directory
for folder in "$SAFE_DIR"/*.SAFE; do
    echo "Checking: $folder"

    # Find all TIFF files inside measurement/ subdir
    tiff_files=$(find "$folder/measurement/" -name "*.tiff")

#!/bin/bash

echo "📂 Checking TIFF and XML files in: $PWD"

# Check TIFF files
for tif in $(find "$PWD" -maxdepth 1 -name "*.tiff"); do
    echo -n "🔍 Checking TIFF: $(basename "$tif") ... "
    if gdalinfo "$tif" &>/dev/null; then
        echo "✅ OK"
    else
        echo "❌ CORRUPTED"
    fi
done

# Check XML files
for xml in $(find "$PWD" -maxdepth 1 -name "*.xml"); do
    echo -n "🔍 Checking XML: $(basename "$xml") ... "
    if grep -q "<product>" "$xml" &>/dev/null; then
        echo "✅ OK"
    else
        echo "❌ CORRUPTED or INVALID"
    fi
done

done """
import os
import subprocess
import glob
import shutil

# Path to your .SAFE folders
base_dir = "/mnt/data/gmtsar_test_1/data"
safe_dirs = glob.glob(os.path.join(base_dir, "*.SAFE"))

for safe in safe_dirs:
    print(f"🔍 Checking: {safe}")
    corrupted = False
    measurement_dir = os.path.join(safe, "measurement")
    tiffs = glob.glob(os.path.join(measurement_dir, "*.tiff"))

    for tiff in tiffs:
        result = subprocess.run(["gdalinfo", tiff],
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE)
        if result.returncode != 0:
            print(f"❌ Corrupted TIFF found: {tiff}")
            #print(f"➡️  Deleting entire folder: {safe}")
            #shutil.rmtree(safe)
            corrupted = True
            break

if not corrupted:
    print("✅ All .SAFE folders checked and clean.")


Some summaries of metadata

In [29]:
search_df_summary = (
    search_df
    .groupby(['pathNumber', 'flightDirection'])
    .size()
    .reset_index(name='scene_count')
    .sort_values(by='scene_count')
)

print(search_df_summary)


   pathNumber flightDirection  scene_count
1         168      DESCENDING           81
0          95      DESCENDING           83


In [33]:
stack_season_summary = (
    stack_season
    .groupby(['pathNumber', 'flightDirection'])
    .size()
    .reset_index(name='scene_count')
    .sort_values(by='scene_count')
)

print("Stack Season Summary:")
print(stack_season_summary)


Stack Season Summary:
   pathNumber flightDirection  scene_count
0         168      DESCENDING           41


In [85]:
# Convert to DataFrame
sbas_df = pd.DataFrame(list(sbas_pairs), columns=["reference", "secondary"])

# Preview
print(sbas_df.head())


                                           reference  \
0  S1A_IW_SLC__1SDV_20240227T052756_20240227T0528...   
1  S1A_IW_SLC__1SDV_20240403T052757_20240403T0528...   
2  S1A_IW_SLC__1SDV_20230127T052752_20230127T0528...   
3  S1A_IW_SLC__1SDV_20230220T052751_20230220T0528...   
4  S1A_IW_SLC__1SDV_20240110T052757_20240110T0528...   

                                           secondary  
0  S1A_IW_SLC__1SDV_20250305T052749_20250305T0528...  
1  S1A_IW_SLC__1SDV_20240427T052757_20240427T0528...  
2  S1A_IW_SLC__1SDV_20230220T052751_20230220T0528...  
3  S1A_IW_SLC__1SDV_20230316T052750_20230316T0528...  
4  S1A_IW_SLC__1SDV_20240122T052757_20240122T0528...  


In [86]:
for i, pair in enumerate(sbas_pairs):
    print(f"{i+1}: {pair}")
    if i > 10:  # Stop after 10 to avoid long output
        print("... (truncated)")
        break

# Check number of elements
print(f"Number of pairs: {len(sbas_pairs)}")

# Print one or two elements to see the structure
for pair in list(sbas_pairs)[:3]:
    print(f"Type: {type(pair)}, Length: {len(pair)}, Content: {pair}")


print(f"Type of sbas_pairs: {type(sbas_pairs)}")
first = next(iter(sbas_pairs))
print(f"Type of first element: {type(first)}")
print(f"Length of first element: {len(first)}")
print(f"Contents of first element: {first}")

lengths = set(len(pair) for pair in sbas_pairs)
print(f"Unique tuple lengths in sbas_pairs: {lengths}")


1: ('S1A_IW_SLC__1SDV_20240227T052756_20240227T052823_052740_0661B3_D2A7', 'S1A_IW_SLC__1SDV_20250305T052749_20250305T052816_058165_072F82_A82A')
2: ('S1A_IW_SLC__1SDV_20240403T052757_20240403T052824_053265_0674B9_97E5', 'S1A_IW_SLC__1SDV_20240427T052757_20240427T052824_053615_06828C_0B08')
3: ('S1A_IW_SLC__1SDV_20230127T052752_20230127T052819_046965_05A203_59A8', 'S1A_IW_SLC__1SDV_20230220T052751_20230220T052818_047315_05ADC6_6AC4')
4: ('S1A_IW_SLC__1SDV_20230220T052751_20230220T052818_047315_05ADC6_6AC4', 'S1A_IW_SLC__1SDV_20230316T052750_20230316T052817_047665_05B99A_EC1E')
5: ('S1A_IW_SLC__1SDV_20240110T052757_20240110T052824_052040_064A04_BE7A', 'S1A_IW_SLC__1SDV_20240122T052757_20240122T052824_052215_064FE6_7F94')
6: ('S1A_IW_SLC__1SDV_20240403T052757_20240403T052824_053265_0674B9_97E5', 'S1C_IW_SLC__1SDV_20250416T052645_20250416T052712_001914_003B88_BA68')
7: ('S1A_IW_SLC__1SDV_20240110T052757_20240110T052824_052040_064A04_BE7A', 'S1A_IW_SLC__1SDV_20250104T052751_20250104T052818

Functions to download the DEM 

In [36]:
def print_aster_wget_instructions(lat_tile='N46', lon_tile='E008'):
    """
    Print instructions and the exact wget command for downloading an ASTER GDEM tile.
    
    Args:
        lat_tile (str): Latitude tile name, e.g., 'N46'
        lon_tile (str): Longitude tile name, e.g., 'E008'
    """
    date_path = "2020.01.01"
    base_url = f"https://e4ftl01.cr.usgs.gov/ASTT/ASTGTM.003/{date_path}"
    tile_name = f"ASTGTMV003_{lat_tile}{lon_tile}_dem.tif"

    print("ASTER GDEM Download Instructions")
    print("1. Make sure you have a ~/.netrc file with your NASA Earthdata login credentials:")
    print("""
machine urs.earthdata.nasa.gov
login your_earthdata_username
password your_earthdata_password
""")
    print("2. Then run the following command in your shell:\n")
    print(f"""wget --load-cookies ~/.urs_cookies \\
     --save-cookies ~/.urs_cookies \\
     --keep-session-cookies \\
     --auth-no-challenge=on \\
     --user-agent="Mozilla/5.0" \\
     --netrc -c {base_url}/{tile_name}\n""")
    print("Tile:", tile_name)
    print("Output: Downloaded .tif file for the specified ASTER tile.")

# Example usage
print_aster_wget_instructions('N46', 'E008')


ASTER GDEM Download Instructions
1. Make sure you have a ~/.netrc file with your NASA Earthdata login credentials:

machine urs.earthdata.nasa.gov
login your_earthdata_username
password your_earthdata_password

2. Then run the following command in your shell:

wget --load-cookies ~/.urs_cookies \
     --save-cookies ~/.urs_cookies \
     --keep-session-cookies \
     --auth-no-challenge=on \
     --user-agent="Mozilla/5.0" \
     --netrc -c https://e4ftl01.cr.usgs.gov/ASTT/ASTGTM.003/2020.01.01/ASTGTMV003_N46E008_dem.tif

Tile: ASTGTMV003_N46E008_dem.tif
Output: Downloaded .tif file for the specified ASTER tile.


In [37]:
import subprocess
from tqdm import tqdm
import os

def download_dem_with_make_dem(W, E, S, N, download_dir, mode=1):
    """
    Download DEM using GMTSAR's make_dem.csh script.

    Parameters:
    - W, E: Western and Eastern longitudes
    - S, N: Southern and Northern latitudes
    - download_dir: directory to save the DEM
    - mode: resolution mode (1 = 1s, 2 = 3s), default is 1
    """
    print(f"[INFO] Downloading DEM for W:{W} E:{E} S:{S} N:{N} with mode {mode}")
    print(f"[INFO] Output directory: {download_dir}")
    
    os.makedirs(download_dir, exist_ok=True)

    cmd = ["make_dem.csh", str(W), str(E), str(S), str(N), str(mode)]
    
    with tqdm(total=1, desc="DEM Download (make_dem.csh)") as pbar:
        subprocess.run(cmd, cwd=download_dir, check=True)
        pbar.update(1)


In [38]:
import os
from pathlib import Path
def download_earth_relief_dem(minlat, maxlat, minlon, maxlon, resolution='03s', download_dir='raw/dem/'):
    """
    Download DEM using GMT's @earth_relief.

    Args:
        minlat, maxlat, minlon, maxlon: float, bounding box
        resolution: str, e.g. '03s', '15s', etc.
        download_dir: str, output directory (default: 'raw/dem/')
    """
    
    Path(download_dir).mkdir(parents=True, exist_ok=True)
    print("GMT earth_relief DEM download:")
    print(f"cd {download_dir}")
    print(f"gmt grdcut @earth_relief_{resolution} -R{minlon}/{maxlon}/{minlat}/{maxlat} -Gdem_relief.grd")

    # Optional: auto-download with subprocess
    # subprocess.run(['gmt', 'grdcut', f'@earth_relief_{resolution}',
    #                 f'-R{minlon}/{maxlon}/{minlat}/{maxlat}', '-Gdem_relief.grd'],
    #                cwd=download_dir)


In [ ]:
# This one uses ISCE2 dem.py 
import subprocess

cmd = [
    "python", "/path/to/isce2/applications/dem.py",
    "-a", "SRTM1",
    "-b", "44.1 47.2 6.2 10.5",
    "-r", "1",
    "-f", "dem_wide"
]

subprocess.run(cmd, check=True)


In [51]:
import math 

topobox = [
    math.floor(bbox[0] - 0.5),  # W
    math.floor(bbox[1] - 0.5),  # S
    math.ceil(bbox[2] + 0.5),  # E
    math.ceil(bbox[3] + 0.5)   # N
]
print("The STACK AOI Bounding Box (W, S, E, N):", bbox)
print("Extended DEM Bounding Box (W, S, E, N):", topobox)

The STACK AOI Bounding Box (W, S, E, N): [8.1619, 42.7625, 12.1348, 46.1562]
Extended DEM Bounding Box (W, S, E, N): [7, 42, 13, 47]


In [52]:
# === Download DEM using GMTSAR's make_dem.csh ===
#from tqdm.notebook import tqdm
from tqdm import tqdm
import subprocess

print("\n Downloading DEM using make_dem.csh...")


# for _ in tqdm(range(1), desc="DEM Download"):
#     download_dem_with_make_dem(
#         W=bbox[0],
#         E=bbox[2],
#         S=bbox[1],
#         N=bbox[3],
#         download_dir=dem_dir,
#         mode=1
#     )

for _ in tqdm(range(1), desc="DEM Download"):
    download_dem_with_make_dem(
        topobox[0],
        topobox[2],
        topobox[1],
        topobox[3],
        download_dir=dem_dir,
        mode=2
    )



DEM Download:   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Downloading DEM for W:7 E:13 S:42 N:47 with mode 2
[INFO] Output directory: /mnt/data/gmtsar_test_2/topo



START: make_dem.csh



grdblend [NOTICE]: Remote data courtesy of GMT data server oceania [http://oceania.generic-mapping-tools.org]

grdblend [NOTICE]: Earth Relief at 3x3 arc seconds tiles provided by SRTMGL3 (land only) [NASA/USGS].
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N42E012
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N43E012
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N44E012
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N45E012
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N46E012
DEM Download: 100%|██████████| 1/1 [00:59<00:00, 59.02s/it]


created dem.grd, heights relative to WGS84 ellipsoid

END: make_dem.csh



##NOW TESTING AGAIN IN A NEW DIRECTORY BASED ON https://gmtsar.github.io/documentation/S1_Batch_Preprocessing.html

In [ ]:
from pathlib import Path

# Define root
root_data = Path("/mnt/data")
project_dir = root_data / "gmtsar_test_2"

# Define top-level directories
top_dirs = ["data", "orbit", "reframed", "topo", "F1", "F2", "F3", "SBAS", "merge"]

# Define F* subdirectories
f_subdirs = ["raw", "SLC", "intf_in", "topo"]

# Create top-level dirs
for d in top_dirs:
    (project_dir / d).mkdir(parents=True, exist_ok=True)

# Create F1/F2/F3 substructure
for f in ["F1", "F2", "F3"]:
    for sub in f_subdirs:
        (project_dir / f / sub).mkdir(parents=True, exist_ok=True)

# Touch the batch_tops.config file inside each F*
for f in ["F1", "F2", "F3"]:
    config_file = project_dir / f / "batch_tops.config"
    config_file.touch()

print(f"Directory structure created under: {project_dir}")


Directory structure created under: /mnt/data/gmtsar_test_2


In [74]:
!tree -d -L 3 /mnt/data/gmtsar_test_2


/mnt/data/gmtsar_test_2
├── F1
│   ├── SLC
│   ├── intf_in
│   ├── raw
│   └── topo
├── F2
│   ├── SLC
│   ├── intf_in
│   ├── raw
│   └── topo
├── F3
│   ├── SLC
│   ├── intf_in
│   ├── raw
│   └── topo
├── SBAS
├── data
├── merge
├── orbit
├── raw
├── reframed
└── topo
    └── mydem
        ├── 00
        ├── 01
        ├── 02
        ├── 03
        └── 04

28 directories


Unzipping SAFE files 

In [88]:
import os
import zipfile
from tqdm.notebook import tqdm

# Loop through ZIPs with tqdm progress bar
for fname in tqdm(os.listdir(data_dir), desc="Unzipping SAFE files"):
    if fname.endswith(".zip"):
        zip_path = os.path.join(data_dir, fname)
        expected_safe = fname.replace(".zip", ".SAFE")
        expected_safe_path = os.path.join(data_dir, expected_safe)
        manifest_path = os.path.join(expected_safe_path, "manifest.safe")

        if os.path.exists(manifest_path):
            print(f"Already unzipped and complete: {expected_safe}")
            continue

        print(f"Unzipping: {fname}")
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(data_dir)
            print(f"Done unzipping: {expected_safe}")
        except Exception as e:
            print(f"Failed to unzip {fname}: {e}")


Unzipping SAFE files:   0%|          | 0/83 [00:00<?, ?it/s]

Already unzipped and complete: S1C_IW_SLC__1SDV_20250510T052647_20250510T052714_002264_004CDC_B596.SAFE
Already unzipped and complete: S1A_IW_SLC__1SDV_20230220T052751_20230220T052818_047315_05ADC6_6AC4.SAFE
Already unzipped and complete: S1A_IW_SLC__1SDV_20250504T052750_20250504T052817_059040_0752C4_8948.SAFE
Already unzipped and complete: S1A_IW_SLC__1SDV_20250317T052749_20250317T052816_058340_073676_E9BD.SAFE
Already unzipped and complete: S1A_IW_SLC__1SDV_20250305T052749_20250305T052816_058165_072F82_A82A.SAFE
Already unzipped and complete: S1A_IW_SLC__1SDV_20230409T052752_20230409T052819_048015_05C56B_325E.SAFE
Already unzipped and complete: S1A_IW_SLC__1SDV_20230503T052752_20230503T052819_048365_05D12D_4D4F.SAFE
Already unzipped and complete: S1A_IW_SLC__1SDV_20240203T052756_20240203T052823_052390_0655D1_FAA4.SAFE
Already unzipped and complete: S1A_IW_SLC__1SDV_20240122T052757_20240122T052824_052215_064FE6_7F94.SAFE
Already unzipped and complete: S1A_IW_SLC__1SDV_20230421T052751_

In [99]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, box
from pathlib import Path
# Define output directory and filename
kml_dir = Path.home() / "work" / "gmtsar_test_2" / "graphics"
kml_dir.mkdir(parents=True, exist_ok=True)
kml_path = kml_dir / "sentinel1_frames.kml"





Make kml of the frames to have an overview of the area for stiching pins selection

In [98]:
print(project_dir)

/mnt/data/gmtsar_test_2


In [96]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape
# import ast

# Load the metadata CSV
# stack = pd.read_csv(project_dir / "sbas_stack_metadata.csv")

# Convert the string in the 'geometry' column to real dicts
# stack["geometry_dict"] = stack["geometry"].apply(ast.literal_eval)
# Convert 'geometry' (already a dict) directly to Shapely

stack['geometry_obj'] = stack['geometry'].apply(shape)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(stack, geometry="geometry_obj", crs="EPSG:4326")

# Print bounding box and preview
print("Number of frames:", gdf["frameNumber"].nunique())
print("Frame numbers:", gdf["frameNumber"].unique())
print("Bounding Box:", gdf.total_bounds)  # [minx, miny, maxx, maxy]

# Optional: Save GeoJSON
gdf[['frameNumber', 'geometry_obj']].to_file(os.path.join(project_dir, "frames.geojson"), driver="GeoJSON")



Number of frames: 3
Frame numbers: [444 446 442]
Bounding Box: [ 8.161853 42.762455 12.134823 46.156193]


In [100]:
# Ensure correct CRS and geometries
gdf["geometry"] = gdf["geometry_obj"]  # Use parsed geometry column
gdf = gdf.set_geometry("geometry")
gdf = gdf.set_crs(epsg=4326)

# Keep only one geometry per frame
unique_frames_gdf = gdf.drop_duplicates(subset="frameNumber")[["frameNumber", "geometry"]].copy()
unique_frames_gdf["name"] = "Frame_" + unique_frames_gdf["frameNumber"].astype(str)

# Define AOI
aoi_box = box(7.6172, 46.2195, 8.0172, 46.6195)
aoi_gdf = gpd.GeoDataFrame([{"name": "aoi_search", "geometry": aoi_box}], crs="EPSG:4326")

# Merge frame polygons and AOI
combined_gdf = pd.concat([unique_frames_gdf[["name", "geometry"]], aoi_gdf], ignore_index=True)

# Define output path
kml_dir.parent.mkdir(parents=True, exist_ok=True)

# Export to KML
combined_gdf.to_file(kml_path, driver="KML")
print(f"✅ Clean KML exported to: {kml_path}")


✅ Clean KML exported to: /home/ubuntu/work/gmtsar_test_2/graphics/sentinel1_frames.kml


In [105]:
from pathlib import Path

# Replace with actual (lon, lat) coordinates
pin1 = (9.990311, 45.444649)  # Northern point
pin2 = (10.320525, 44.919137)  # 
pin3 = (10.080915, 44.294765)  # 
pin4 = (10.517160, 43.738494)  # 
#pin5 = (10.806525, 43.027770)  # Southern point



pins_ll = Path(project_dir) / "reframed" / "pins.ll"
pins_ll.parent.mkdir(exist_ok=True)  # Ensure reframed/ exists
with open(pins_ll, "w") as f:
    f.write(f"{pin1[0]} {pin1[1]}\n")
    f.write(f"{pin2[0]} {pin2[1]}\n")
    f.write(f"{pin3[0]} {pin3[1]}\n")
    f.write(f"{pin4[0]} {pin4[1]}\n")
   # f.write(f"{pin5[0]} {pin5[1]}\n")

print(f"Written to: {pins_ll}")


Written to: /mnt/data/gmtsar_test_2/reframed/pins.ll


Downloading precise orbit using sentineleof 0.11.0

In [75]:
##Need to instal: pip install --upgrade sentineleof ver 0.11.0 for S1C

# === Download orbits for all SLCs under /raw ===

print("\n Searching for SLC path folders to fetch orbits...")
#from tqdm.notebook import tqdm
from tqdm import tqdm
paths = glob.glob(os.path.join(orbit_dir, "path_*"))

for orbit_dir in tqdm(paths, desc="Orbit Download", unit="path"):
    tqdm.write(f"Downloading orbits for: {download_dir}")
    subprocess.run([
        "eof",
        "--search-path", download_dir,
        "--save-dir", orbit_dir,
        "--force-asf"
    ], check=True)



 Searching for SLC path folders to fetch orbits...


Orbit Download: 0path [00:00, ?path/s]


In [108]:
from pathlib import Path
from datetime import datetime
import re

frames = {
    "F1": "iw1",
    "F2": "iw2",
    "F3": "iw3",
}

def extract_datetime_from_tiff(tiff_name):
    match = re.search(r"-(\d{8}t\d{6})-", tiff_name.lower())
    if not match:
        return None
    try:
        return datetime.strptime(match.group(1), "%Y%m%dt%H%M%S")
    except:
        return None

def orbit_covers(eof_name, dt):
    try:
        parts = eof_name.split("_V")[1].split("_")
        start = datetime.strptime(parts[0], "%Y%m%dT%H%M%S")
        end   = datetime.strptime(parts[1].replace(".EOF", ""), "%Y%m%dT%H%M%S")
        return start <= dt <= end
    except:
        return False

# Loop through each frame
for frame, iw in frames.items():
    raw_dir = Path(project_dir) / frame / "raw"
    tiffs = sorted(raw_dir.glob(f"*{iw}*vv*.tiff")) + sorted(raw_dir.glob(f"*{iw}*vh*.tiff"))
    eofs = sorted(Path(data_dir).glob("*.EOF"))
    data_in = raw_dir / "data.in"

    count = 0
    with open(data_in, "w") as f:
        for tiff in tiffs:
            dt = extract_datetime_from_tiff(tiff.name)
            if not dt:
                print(f" Skipping {tiff.name} (no datetime found)")
                continue

            match = None
            for eof in eofs:
                if orbit_covers(eof.name, dt):
                    match = eof.name
                    break

            if match:
                f.write(f"{tiff.stem}:{match}\n")
                count += 1
            else:
                print(f" No matching EOF for {tiff.name}")

    print(f"✅ Created {data_in} with {count} entries.")


✅ Created /mnt/data/gmtsar_test_2/F1/raw/data.in with 82 entries.
✅ Created /mnt/data/gmtsar_test_2/F2/raw/data.in with 82 entries.
✅ Created /mnt/data/gmtsar_test_2/F3/raw/data.in with 82 entries.


Select a pair before and after an incident 

Get the satellite footpring from LED file 

In [ ]:
import subprocess
from pyproj import Transformer
import numpy as np
import os

# Set your raw directory path (adjust if needed)
raw_dir = "/mnt/data/gmtsar_test_1/F1/raw"
led_files = [f for f in os.listdir(raw_dir) if f.endswith("F1.LED")]

# Transformer from ECEF to lat/lon
transformer = Transformer.from_crs("epsg:4978", "epsg:4326", always_xy=True)

swath_bounds = {}

for led_file in led_files:
    swath = led_file.split("_")[2]  # E.g., 053438
    path = os.path.join(raw_dir, led_file)

    # Extract first and last line ECEF coordinates
    awk_cmd = f"awk 'NR==2{{print $4, $5, $6}} END{{print $4, $5, $6}}' {path}"
    result = subprocess.check_output(awk_cmd, shell=True).decode().strip().split("\n")
    xyz_start = list(map(float, result[0].split()))
    xyz_end = list(map(float, result[1].split()))
    print(f"[Swath {swath}] Start ECEF: {xyz_start}, End ECEF: {xyz_end}")
    # Convert to lat/lon
    lon1, lat1, _ = transformer.transform(xyz_start[0], xyz_start[1], xyz_start[2])
    lon2, lat2, _ = transformer.transform(xyz_end[0], xyz_end[1], xyz_end[2])

    # Get bounding box
    W = round(min(lon1, lon2), 4)
    E = round(max(lon1, lon2), 4)
    S = round(min(lat1, lat2), 4)
    N = round(max(lat1, lat2), 4)

    swath_bounds[swath] = {"W": W, "E": E, "S": S, "N": N}
    print(f"[Swath {swath}] W: {W}, E: {E}, S: {S}, N: {N}")

# === Merge bounding boxes ===
W_all = min(b["W"] for b in swath_bounds.values())
E_all = max(b["E"] for b in swath_bounds.values())
S_all = min(b["S"] for b in swath_bounds.values())
N_all = max(b["N"] for b in swath_bounds.values())

print("\n[Combined Coverage for All Swaths]")
print(f"W: {W_all}, E: {E_all}, S: {S_all}, N: {N_all}")

# === Check DEM coverage using GMT ===
dem_path = "/mnt/data/gmtsar_test_1/F1/topo/dem.grd"
try:
    grdinfo = subprocess.check_output(f"gmt grdinfo {dem_path}", shell=True).decode()
    print("\n[DEM grdinfo Output]")
    print(grdinfo)
except subprocess.CalledProcessError:
    print("[ERROR] Could not read dem.grd — check path or GMT install.")


[Swath ALL] Start ECEF: [-4654596.487311, -223083.269757, 5315068.231733], End ECEF: [5351057.468749, -620367.425822, -4596147.382489]
[Swath ALL] W: -177.2561, E: -6.613, S: -40.6421, N: 48.9294
[Swath ALL] Start ECEF: [-4714040.444577, -247922.081594, 5261442.231315], End ECEF: [5405304.917568, -606441.635012, -4534014.418914]
[Swath ALL] W: -176.9895, E: -6.4015, S: -39.9842, N: 48.2738
[Swath 053438] Start ECEF: [-4654596.487311, -223083.269757, 5315068.231733], End ECEF: [5351057.468749, -620367.425822, -4596147.382489]
[Swath 053438] W: -177.2561, E: -6.613, S: -40.6421, N: 48.9294
[Swath 053538] Start ECEF: [-4714040.444577, -247922.081594, 5261442.231315], End ECEF: [5405304.917568, -606441.635012, -4534014.418914]
[Swath 053538] W: -176.9895, E: -6.4015, S: -39.9842, N: 48.2738

[Combined Coverage for All Swaths]
W: -177.2561, E: -6.4015, S: -40.6421, N: 48.9294

[DEM grdinfo Output]
/mnt/data/gmtsar_test_1/F1/topo/dem.grd: Title: Produced by grdmath
/mnt/data/gmtsar_test_1/F1

In [ ]:
#print("Available columns:\n", stack_season.columns.tolist())
print(stack_season[['sceneName', 'platform', 'temporalBaseline']].head(3))

                                           sceneName     platform  \
0  S1A_IW_SLC__1SDV_20250415T053537_20250415T0536...  Sentinel-1A   
1  S1A_IW_SLC__1SDV_20250427T053538_20250427T0536...  Sentinel-1A   
2  S1C_IW_SLC__1SDV_20250503T053434_20250503T0535...  Sentinel-1C   

   temporalBaseline  
0                 0  
1                12  
2                18  
